# Arqtic Weather Pipeline — Mission Presentation

**Challenge**: Build a data pipeline that pulls weather data, processes it, and creates an interactive dashboard. Deploy to a major cloud provider using Infrastructure as Code.

**Author**: Andre  
**Date**: February 2026  
**Stack**: Python 3.14 · DuckDB · Streamlit · Plotly · Prophet · Terraform · GCP

## 1. Architecture Overview

![Architecture](../architecture.png)

The solution follows a **local-first development model** — everything works on a laptop, then deploys to GCP as the final step.

| Component | Technology | Rationale |
|-----------|------------|----------|
| Weather Data | Open-Meteo Forecast + Archive API | Free, no API key, WMO-backed accuracy |
| Air Quality | Open-Meteo Air Quality API | Same provider, free, US AQI + PM2.5 hourly |
| Data Tool | DuckDB + Parquet | Embedded SQL engine, zero-config, transparent GCS reads via httpfs |
| Quality Gate | Pandera (11 validated fields) | DataFrame schema validation, catches bad data before storage |
| Dashboard | Streamlit + Plotly | Interactive, Pythonic, drill-down with range selectors |
| Forecasting | Prophet | Robust time-series with yearly seasonality and uncertainty bands |
| IaC | Terraform | GCS, Cloud Run, Scheduler, IAM — all version-controlled |
| CI/CD | GitHub Actions | Lint + test on PR, build + deploy on merge |
| Package Manager | uv | 10x faster than pip, deterministic lockfile |

## 2. Data Pipeline — Live Demo

The pipeline runs in 4 stages: **Extract → Validate → Transform → Load**

In [ ]:
import os
import sys

sys.path.insert(0, os.path.join(os.getcwd(), ".."))

# Run the full pipeline
from pipeline.run import main

main()

## 3. Data Quality Gate

Every row passes through Pandera schema validation before it reaches storage. Invalid data (temperatures > 60°C, negative wind speeds, humidity > 100%) is **rejected immediately**.

**Critical discovery**: On Python 3.14, `import pandera as pa` fails with a `KeyError`. The correct import is `import pandera.pandas as pa`. This is undocumented — found through systematic testing.

In [ ]:
from pipeline.quality import DailyWeatherSchema

# Show the schema definition
print("Daily Weather Schema:")
print(DailyWeatherSchema)

In [ ]:
import pandas as pd

# Demonstrate the quality gate catching bad data
bad_data = pd.DataFrame(
    [
        {
            "date": pd.Timestamp("2025-06-15"),
            "temperature_2m_max": 999.0,  # Impossible!
            "temperature_2m_min": 15.0,
            "apparent_temperature_max": 27.0,
            "apparent_temperature_min": 13.0,
            "precipitation_sum": 2.5,
            "wind_speed_10m_max": 15.0,
            "relative_humidity_2m_mean": 65.0,
            "weather_code": 3.0,
            "daylight_duration": 52000.0,
        }
    ]
)

try:
    DailyWeatherSchema.validate(bad_data, lazy=True)
except Exception as e:
    print(f"Quality gate rejected: {type(e).__name__}")
    print("Reason: temperature_2m_max=999.0 exceeds range [-60, 60]")

## 4. Transformation Layer — Making Weather Data Useful

Raw weather numbers are meaningless to most people. The transform stage adds:

1. **Thermal comfort labels**: Maps `apparent_temperature` to UTCI stress categories, then translates to everyday language: *"Cold — Wear layers and a warm jacket."*
2. **WMO weather descriptions**: Code 73 → "Moderate snow ❄️"
3. **Wind gust labels**: Speed + gust thresholds → "Windy — secure loose items" or `None` when calm (silence = safe)
4. **Visibility labels**: Meters → "Low visibility — fog" or `None` when clear
5. **Anomaly detection**: Z-score against 30-day rolling mean
6. **Historical comparison**: Today's temp vs multi-year average for this calendar date
7. **Daylight hours**: Duration converted for clean display

### Why apparent_temperature instead of pythermalcomfort?

We investigated `pythermalcomfort` for scientifically precise UTCI calculations. However:
- Open-Meteo's `apparent_temperature` already factors in wind chill and humidity
- Mapping it to UTCI stress thresholds produces equivalent comfort categories
- This removes 4 heavy transitive dependencies (numba, llvmlite, scipy, pythermalcomfort)
- Docker image shrinks significantly; no LLVM compilation during build

**Result**: Same user-facing outcome, simpler stack, faster builds.

### Supplementary data sources (best-effort pattern)

The pipeline also fetches two supplementary datasets using direct HTTP calls (bypassing the SDK):
- **Sunrise/sunset times**: The Open-Meteo SDK can't deserialize ISO8601 string fields. Direct JSON parsing works. Used for accurate sunset countdown in the dashboard.
- **Air quality (US AQI, PM2.5)**: Same Open-Meteo provider, free, no API key. Enables health-relevant alerts in the dashboard.

Both are **best-effort** — if either API call fails, the pipeline continues and the dashboard gracefully degrades (those alerts simply don't appear).

In [ ]:
from pipeline.transform import COMFORT_TRANSLATIONS, _get_stress_category

# Show the thermal comfort mapping
test_temps = [-45, -35, -20, -5, 5, 15, 28, 35, 42, 50]

print(f"{'Apparent °C':>12} | {'Stress Category':<28} | {'Everyday Label':<18} | Advice")
print("-" * 100)
for t in test_temps:
    cat = _get_stress_category(t)
    label, advice, color = COMFORT_TRANSLATIONS[cat]
    print(f"{t:>12.0f} | {cat:<28} | {color} {label:<15} | {advice}")

## 5. DuckDB — The Right-Sized Data Tool

DuckDB was chosen over BigQuery, Snowflake, or PostgreSQL because:
- **Embedded**: No server, no credentials, zero-config
- **Parquet-native**: Reads Parquet files directly with columnar pushdown
- **GCS-transparent**: `httpfs` extension reads `gs://` paths in production
- **SQL power**: Aggregation, window functions, and joins without loading full DataFrames

In [ ]:
import duckdb

# Query weather data directly from Parquet with SQL
result = duckdb.query("""
    SELECT 
        condition_text,
        COUNT(*) as days,
        ROUND(AVG(temperature_2m_max), 1) as avg_high,
        ROUND(AVG(temperature_2m_min), 1) as avg_low
    FROM read_parquet('../data/daily/weather.parquet')
    GROUP BY condition_text
    ORDER BY days DESC
    LIMIT 10
""").df()

print("Weather condition distribution (3 years):")
result

## 6. Dashboard Highlights

The Streamlit dashboard has 4 tabs:

| Tab | Purpose | Key Features |
|-----|---------|-------------|
| Right Now | 3-second glanceable view | Hero display, smart alerts, Morning/Afternoon/Evening breakdown, weekly outlook |
| Trends | Interactive time-series | Plotly with range selectors, anomaly markers, weekly/monthly aggregation via DuckDB |
| Forecast | 30-day Prophet prediction | Uncertainty bands, comfort labels for predicted temperatures |
| Data Quality | Pipeline health | Freshness indicator, validation status, anomaly log, data statistics |

### "Right Now" Tab — Decision-Oriented Design

The primary tab is designed for someone opening the dashboard at 7am before leaving the house. Instead of showing raw numbers, it synthesizes data into **actionable decisions**:

- **Hero section**: Current condition, feels-like temperature, comfort advice, historical comparison as a subtle subtitle
- **Smart alerts** (only appear when relevant — silence means safe):
  - Precipitation: *"Bring layers — light snow likely this morning"* (uses temperature to distinguish rain vs snow)
  - Visibility: *"Low visibility this morning — drive carefully"*
  - Air quality: *"Air quality is moderate (AQI 75) — sensitive groups should limit outdoor activity"*
  - UV: *"High UV (8) — wear sunscreen"*
  - Sunset: *"Sunset at 5:40pm (in ~11h 37m)"* (uses real sunrise/sunset data)
- **Your Day**: Morning (6-12), Afternoon (12-5), Evening (5-10) — each shows average temp, feels-like, dominant condition icon, comfort advice, and precipitation note if > 20% probability
- **This Week**: 7-day outlook with comfort labels instead of low temperatures: *"Wed ❄️ -7° · Cold"*

### Design Decisions
- **`st.fragment`** on Forecast tab: Prophet training (~3s) only reruns when that specific tab is active
- **`st.cache_data(ttl=600)`**: Parquet reads cached for 10 minutes
- **Silence = safe**: Alerts only appear when action is needed. No wind alert today? Wind is fine.
- **Temperature-based precipitation type**: If average temp ≤ 2°C, say "snow" not "rain" — more accurate than weather code mode

Run the dashboard: `make run-dashboard` → http://localhost:8501

## 7. Prophet Forecasting — Live Demo

In [ ]:
import pandas as pd

from forecast.predict import make_forecast

# Load daily data and run forecast
daily = pd.read_parquet("../data/daily/weather.parquet")
forecast = make_forecast(daily, metric_col="temperature_2m_max", periods=30)

# Show the next 7 days
future = forecast[forecast["ds"] > daily["date"].max()].head(7)
print("Next 7 days forecast:")
for _, row in future.iterrows():
    cat = _get_stress_category(row["yhat"])
    label = COMFORT_TRANSLATIONS[cat][0]
    print(
        f"  {row['ds'].strftime('%a %b %d')}: {row['yhat']:.1f}°C ({label}) "
        f"[{row['yhat_lower']:.1f} to {row['yhat_upper']:.1f}]"
    )

## 8. Infrastructure as Code (Terraform)

All GCP resources are managed by Terraform:

```
terraform/
├── provider.tf           # GCP provider + GCS backend for remote state
├── variables.tf          # Configurable inputs (project, region, location)
├── storage.tf            # GCS bucket for weather data
├── artifact_registry.tf  # Docker image registry
├── pipeline.tf           # Cloud Run Job + Cloud Scheduler
├── dashboard.tf          # Cloud Run Service (public) + startup probe
├── iam.tf                # Least-privilege service account
├── outputs.tf            # Dashboard URL, bucket name
└── tests/
    └── main.tftest.hcl   # Native Terraform tests (plan-mode validation)
```

### Key decisions:
- **Cloud Run** over GKE: Right-sized for a single dashboard + periodic job
- **Cloud Scheduler** over Cloud Composer: No need for a full Airflow instance for one cron job
- **GCS** over BigQuery: Parquet files are the perfect abstraction for this data volume
- **Remote state** in GCS: Team-safe Terraform state management
- **Native `tftest.hcl`**: Tests Terraform logic without deploying resources

## 9. SDLC, DataOps & DataQuality Practices

The assessment asks for **at least one** — we implemented all three:

### SDLC
- Git flow: `main` + feature branches
- Pre-commit hooks: ruff lint/format + terraform fmt/validate
- CI: Automated lint + test on every PR
- CD: Build + deploy on merge to main

### DataOps
- Scheduled pipeline execution (Cloud Scheduler, every 6 hours)
- Environment-driven config (`DATA_PATH` switches local ↔ cloud)
- Deterministic builds (`uv.lock`)
- Multi-stage Docker for minimal images

### DataQuality
- **Quality gate**: Pandera schema validation — 11 hourly fields and 10 daily fields checked before data reaches storage
- **Anomaly detection**: Z-score against 30-day rolling mean, visible in dashboard
- **Freshness monitoring**: File modification time tracked in Data Quality tab
- **Null tracking**: Zero nulls confirmed in data statistics
- **Graceful degradation**: Supplementary data (AQI, sun times) is best-effort — pipeline and dashboard continue if those sources are unavailable

## 10. Testing

In [ ]:
import subprocess

result = subprocess.run(
    ["python", "-m", "pytest", "../tests/", "-v", "--tb=short"], capture_output=True, text=True, cwd=".."
)
print(result.stdout)
if result.returncode != 0:
    print(result.stderr)

## 11. Production Considerations

| Area | Current | Production Enhancement |
|------|---------|----------------------|
| Auth | SA key in GitHub secret | Workload Identity Federation (OIDC) |
| Monitoring | Data Quality tab | Cloud Monitoring alerts on pipeline failures |
| Scaling | Single Cloud Run instance | Autoscaling with min instances for cold start |
| Multi-location | Single city (Toronto) | Config-driven, one pipeline per city |
| Data retention | 90-day GCS lifecycle | Archival to Coldline for historical analysis |
| Secret management | Environment variables | Secret Manager integration |

## 12. Cost Considerations

| Resource | Usage Pattern | Cost Expectation |
|----------|--------------|-----------------|
| Cloud Run Service | Low-traffic dashboard, scales to zero | Minimal (within free tier) |
| Cloud Run Job | 4 short runs/day | Minimal (within free tier) |
| GCS Storage | Small Parquet files | Minimal (within free tier) |
| Artifact Registry | Single Docker image | Negligible |
| Cloud Scheduler | 4 triggers/day | Free (3 free jobs included) |

The solution was deliberately designed to fit within GCP's free tier — all components are right-sized to avoid unnecessary cost while maintaining full functionality.

## Summary

This solution demonstrates:
- **End-to-end pipeline**: Extract → Validate → Transform → Load → Visualize (4 data sources, 11 validated fields)
- **Practical engineering**: Right-sized tools (DuckDB, not BigQuery), no over-engineering
- **Human-centered design**: Weather data translated to actionable decisions — not raw numbers but "bring an umbrella", "wear layers", "air quality is moderate"
- **Complementary data sources**: Air quality, sunrise/sunset, wind gusts, visibility — all free, all from Open-Meteo, all adding genuine value
- **Quality-first**: Pandera schema validation, z-score anomaly detection, freshness monitoring
- **Production-ready IaC**: Terraform-managed GCP with CI/CD automation, native `.tftest.hcl` tests
- **29 passing tests** (unit + integration E2E), zero lint errors, deterministic builds with `uv.lock`

All code: [github.com/andreibuaka/arqtic](https://github.com/andreibuaka/arqtic)